In [47]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
from Robot_Bistrot.gepetto import Robot_bistro

In [46]:
# Clé API Google Places
with open('apikeyrm.txt', 'r') as file: #mettre clé api
    API_KEY = file.read().strip()

RADIUS = 5000  # en mètres
QUERY = 'pizza budget 10 euros toulouse capitole'


url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={QUERY}&key={API_KEY}&maxResults=10"

response = requests.get(url)
data = response.json()


nb_result = 10

#Utilisation de compréhension de dictionnaire pour créer le dictionnaire
dict_adresses = {
    'restaurant_id' : [result['name'] + str(result['geometry']['location']['lat']) for result in data['results'][:nb_result]],
    'name' : [result['name'] for result in data['results'][:nb_result]],
    'formatted_address': [result['formatted_address'] for result in data['results'][:nb_result]],
    'lat': [result['geometry']['location']['lat'] for result in data['results'][:nb_result]],
    'lng' : [result['geometry']['location']['lng'] for result in data['results'][:nb_result]],
    'photo_reference' : [result['photos'][0]['photo_reference'] for result in data['results'][:nb_result]],
    'rating': [result['rating'] for result in data['results'][:nb_result]],
    'user_ratings_total' : [result['user_ratings_total'] for result in data['results'][:nb_result]],
    'types': [result['types'] for result in data['results'][:nb_result]]
}

#Affichage du dictionnaire pour vérification
print(dict_adresses)

{'restaurant_id': ['PL Pizza Léo43.6075228', "La Grand' Pizzeria43.6048929", 'Pizza Lopez CARMES43.5977082', 'Bar Pizzeria Osteria Le Bellini43.6039846', 'Pizza Cap Carmes43.5964068', 'Mazzolina Pizzeria et Restaurant43.6045329', 'La Pastasciutta43.6062834', 'Marcello Saint-Pierre43.6040015', 'Toto Bello Pizzas43.6024277', 'PIZZA CAP43.6062683'], 'name': ['PL Pizza Léo', "La Grand' Pizzeria", 'Pizza Lopez CARMES', 'Bar Pizzeria Osteria Le Bellini', 'Pizza Cap Carmes', 'Mazzolina Pizzeria et Restaurant', 'La Pastasciutta', 'Marcello Saint-Pierre', 'Toto Bello Pizzas', 'PIZZA CAP'], 'formatted_address': ['16 Rue Emile Cartailhac, 31000 Toulouse, France', '4 Rue Castellane, 31000 Toulouse, France', '2 Pl. des Carmes, 31000 Toulouse, France', '9 Pl. du Capitole, 31000 Toulouse, France', '26 Rue du Languedoc, 31000 Toulouse, France', '14-16 Rue de la Colombette, 31000 Toulouse, France', '35Bis Rue Gabriel Péri, 31000 Toulouse, France', '2 Rue Pargaminières, 31000 Toulouse, France', '23 Bd J

In [41]:
df = pd.DataFrame(dict_adresses)

In [42]:
# Informations de connexion
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
HOST = 'localhost'
PORT = 5432
USER = 'postgres'
PASSWORD = 'Legnome13$'
DATABASE = 'search_and_go'

# Chaîne de connexion
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")


In [43]:
# Convertir la colonne "name" en liste
id_list = df["restaurant_id"].tolist()

# Générer les paramètres dynamiquement (:name0, :name1, etc.)
param_dict = {f"restaurant_id{i}": id for i, id in enumerate(id_list)}
placeholders = ", ".join(f":restaurant_id{i}" for i in range(len(id_list)))

# Requête SQL avec IN
query = text(f"SELECT * FROM restaurants WHERE restaurant_id IN ({placeholders})")

# Exécution de la requête
with engine.connect() as connection:
    result = connection.execute(query, param_dict)
    restaurants = result.fetchall()

restaurants = pd.DataFrame(restaurants)
if len(restaurants) != 0:
    restaurants = set(restaurants["restaurant_id"].unique())  # Convertir en ensemble d'IDs
df_push = df.loc[~df["restaurant_id"].isin(restaurants)]  # Filtrer correctement

In [44]:
df_push.to_sql(name='restaurants', con=engine, if_exists = 'append', index=False)

0

In [48]:
bot = Robot_bistro()

In [51]:
bot.talk("ca va")

'Ça va bien, merci ! Et toi ?\n'

In [53]:
test = bot.gethistory()

In [56]:
test[3]

parts {
  text: "Ça va bien, merci ! Et toi ?\n"
}
role: "model"